<a href="https://colab.research.google.com/github/karoldem/comicfinder/blob/master/comicfinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dear reader,

what you see is some unordered fragments of code to be stolen from.

It is for webscraping comics pages, performing OCR on images and saving results to a file.

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

In [1]:
import requests, os, bs4
from io import BytesIO
import pytesseract
import shutil
import os
import random
import json
import operator

try:
    from PIL import Image
except ImportError:
    import Image

savefile = False
if savefile:
    from google.colab import drive
    drive.mount('/content/gdrive')

def appendToJSONFile(newKey, newValue):

    with open('/content/gdrive/My Drive/comictranscriotion.txt', "r") as jsonFile:
        data = json.load(jsonFile)

    data[ newKey ] = newValue
    
    with open('/content/gdrive/My Drive/comictranscriotion.txt', "w") as jsonFile:
        json.dump(data, jsonFile)


class comicOCRer:


    def __init__(self, pageUrl, firstComicURL, lastComicURL, findThoseWordsInText):
        
        self.pageUrl = pageUrl
        self.firstComicURL = firstComicURL
        self.lastComicURL =  lastComicURL
        self.findThoseWordsInText = findThoseWordsInText
        
        if pageUrl == 'http://xkcd.com':
            self.selectNextComic = lambda x : 'http://xkcd.com' + x.select('a[rel="prev"]')[0].get('href')
            self.selectPreviousComic = lambda x : 'http://xkcd.com' + x.select('a[rel="next"]')[0].get('href')
            self.getComicElem = lambda x : x.select('#comic img')
            self.getComicUrl = lambda x: 'http:' + x[0].get('src')
            self.checkLastComic = self.checkLastComicXkcd

        if pageUrl == 'http://smbc-comics.com':
            self.selectNextComic = lambda x : x.select('a[rel="prev"]')[0].get('href')
            self.selectPreviousComic = lambda x : x.select('a[rel="next"]')[0].get('href')
            self.getComicElem = lambda x : x.select('#cc-comicbody img')
            self.getComicUrl = lambda x: x[0].get('src')
            self.checkLastComic = self.checkLastComicXkcd


    def run(self):
        return self.extractText(
                self.pageUrl, 
                self.firstComicURL, 
                self.lastComicURL, 
                self.findThoseWordsInText, 
                self.selectNextComic,
                self.getComicElem)

    turnOffOCR = False
    
    def checkLastComicXkcd(self):
        res = requests.get(self.pageUrl)
        res.raise_for_status()
        soup = bs4.BeautifulSoup(res.text)
        url = self.selectNextComic(soup)
        res = requests.get(url)
        res.raise_for_status()
        soup = bs4.BeautifulSoup(res.text)
        return self.selectPreviousComic(soup)

    def extractText(self, pageUrl, firstComicURL, lastComicURL, findThoseWordsInText, selectNextComic, getComicElem):

      wordCounter = {}
      transcription = {}
      url = firstComicURL

      while not url == lastComicURL:
          comicElem = []
          for i in range(2):
            res = requests.get(url)
            res.raise_for_status()
            soup = bs4.BeautifulSoup(res.text)
            comicElem = self.getComicElem(soup)
            if comicElem != []: break
            
          if comicElem == []:
              wordCounter[url] = 0
          else:
              try:
                    comicUrl = self.getComicUrl(comicElem)
                    res = requests.get(comicUrl)
                    res.raise_for_status()
                    img = Image.open(BytesIO(res.content))

                    if not self.turnOffOCR:
                      comicText = pytesseract.image_to_string(img)
                      if savefile:
                        appendToJSONFile(url,comicText.capitalize())
                      transcription[url] = comicText.capitalize()

                      for i in findThoseWordsInText:
                        if i in comicText:

                          try:
                            wordCounter[url] +=1
                          except:
                            wordCounter[url] = 1
                    else:
                      print (url)

              except Exception as e:
                    wordCounter[url] = 0
                    print(e)


          try: 
            url = selectNextComic(soup)
          except:
              break
      
      
      wordCounter = sorted(wordCounter.items(), key=operator.itemgetter(1), reverse=True)
      return wordCounter

In [3]:
comicOCRer(
    pageUrl = 'http://smbc-comics.com',
    firstComicURL = 'http://smbc-comics.com',
    lastComicURL = 'https://www.smbc-comics.com/comic/i',
    findThoseWordsInText = ['direct', 'cultural', 'some', 'countries']
    ).run()

[]

In [ ]:
comicOCRer(
    pageUrl = 'http://xkcd.com',
    firstComicURL = 'https://xkcd.com',
    lastComicURL = 'hsdfghjkiuytgf',
    findThoseWordsInText = []
    ).run()